In [47]:
import pandas as pd
from transformer.DataAggregator import DataAggregator
from transformer.Dataformator import DataFormator
from transformer.ImputeMean import ImputeMean
from transformer.TrainTestSplit import TrainTestSplit
import utils
import numpy as np
import json

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import math
import statistics

from hyperopt import fmin, hp, tpe, Trials,STATUS_OK
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
# from configs.space import space
from hyperopt.pyll.base import scope

## Data Loading

In [48]:
sales = pd.read_csv("./data/sales_train_validation.csv")
calender =pd.read_csv("./data/calendar.csv")

## Data Transformation

In [49]:
data_aggregator = DataAggregator(sales)
aggregated_data = data_aggregator.aggregate(["store_id","dept_id"],"sum")
data_formator =DataFormator(aggregated_data,calender)
data = data_formator.format_data('store_id','dept_id')
data.index.name='date'
impute_mean =ImputeMean(data)
data = impute_mean.replace_zero_with_mean()
tts = TrainTestSplit( data, test_size=0.3, random_state=0,shuffle=False,stratify=None)
X_train_, X_test_, y_train_, y_test_ = tts.split_data()
y_train_.index.freq,y_test_.index.freq= 'd','d'

## Training for best Parameters Model tuning

### Model Tuning

In [50]:
space = {
    'p': hp.choice('p', range(0, 3)),
    'd': hp.choice('d', range(0, 3)),
    'q': hp.choice('q', range(0, 3)),
    'P': hp.choice('P', range(0, 3)),
    'D': hp.choice('D', range(0, 3)),
    'Q': hp.choice('Q', range(0, 3)),
    'seasonal_order': hp.choice('seasonal_order', [(0,0,0,0), (1,0,0,12), (0,1,0,12), (0,0,1,12), (1,1,0,12), (1,0,1,12), (0,1,1,12), (1,1,1,12)])
}

In [51]:
best ={}



for i in data.columns:
    y_train_data_ = pd.DataFrame()
    y_test_data_ = pd.DataFrame()
    y_train_data_[i] = y_train_[i].astype('float')
    y_test_data_[i] = y_test_[i].astype('float')

    def objective( params):
        sarimax_model = sm.tsa.statespace.SARIMAX(endog=y_train_data_[i], order=(params['p'], params['d'], params['q']), seasonal_order=(params['P'], params['D'], params['Q'], 12))
        sarimax_result = sarimax_model.fit()
        y_pred = sarimax_result.forecast(steps=y_test_data_.shape[0])
        mse = mean_absolute_error(y_test_data_, y_pred)
        return {'loss': mse, 'status': STATUS_OK}

    
    print(f"Training .......\n model_{i} ")

    trials = Trials()
    # optimize hyperparameters using Hyperopt's Tree-structured Parzen Estimator (TPE) algorithm
    best["best_param_{0}".format(i)] = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=3,trials=trials)

    print(best["best_param_{0}".format(i)])

Training .......
 model_CA_1_FOODS_1 
100%|██████████| 3/3 [00:04<00:00,  1.34s/trial, best loss: 81.41000707952641]
{'D': 1, 'P': 0, 'Q': 1, 'd': 1, 'p': 1, 'q': 1, 'seasonal_order': 4}
Training .......
 model_CA_1_FOODS_2 
  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

 67%|██████▋   | 2/3 [00:01<00:00,  1.06trial/s, best loss: 100.43647591632218]

100%|██████████| 3/3 [00:06<00:00,  2.28s/trial, best loss: 100.43647591632218]
{'D': 0, 'P': 0, 'Q': 1, 'd': 1, 'p': 2, 'q': 1, 'seasonal_order': 2}
Training .......
 model_CA_1_FOODS_3 
100%|██████████| 3/3 [00:12<00:00,  4.30s/trial, best loss: 388.54290009493576]
{'D': 1, 'P': 1, 'Q': 2, 'd': 0, 'p': 1, 'q': 1, 'seasonal_order': 7}
Training .......
 model_CA_1_HOBBIES_1 
 67%|██████▋   | 2/3 [00:10<00:05,  5.92s/trial, best loss: 93.35256693585463] 

100%|██████████| 3/3 [00:10<00:00,  3.67s/trial, best loss: 85.41536936279779]
{'D': 0, 'P': 0, 'Q': 0, 'd': 1, 'p': 2, 'q': 1, 'seasonal_order': 7}
Training .......
 model_CA_1_HOBBIES_2 
100%|██████████| 3/3 [00:04<00:00,  1.44s/trial, best loss: 17.967413493068793]
{'D': 0, 'P': 1, 'Q': 1, 'd': 0, 'p': 1, 'q': 1, 'seasonal_order': 1}
Training .......
 model_CA_1_HOUSEHOLD_1 
  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

 33%|███▎      | 1/3 [00:13<00:27, 13.57s/trial, best loss: 155.06694181121358]

...

100%|██████████| 3/3 [00:05<00:00,  1.94s/trial, best loss: 60.2382962210999] 
{'D': 0, 'P': 2, 'Q': 1, 'd': 2, 'p': 1, 'q': 2, 'seasonal_order': 5}
Training .......
 model_WI_3_FOODS_2 
100%|██████████| 3/3 [00:07<00:00,  2.54s/trial, best loss: 141.71394045673762]
{'D': 1, 'P': 0, 'Q': 1, 'd': 0, 'p': 0, 'q': 0, 'seasonal_order': 4}
Training .......
 model_WI_3_FOODS_3 
 33%|███▎      | 1/3 [00:03<00:06,  3.45s/trial, best loss: 333.32574837552175]

100%|██████████| 3/3 [00:42<00:00, 14.31s/trial, best loss: 333.32574837552175]
{'D': 1, 'P': 1, 'Q': 1, 'd': 0, 'p': 0, 'q': 2, 'seasonal_order': 4}
Training .......
 model_WI_3_HOBBIES_1 
 33%|███▎      | 1/3 [00:02<00:05,  2.65s/trial, best loss: 55.09070342715288]

100%|██████████| 3/3 [00:34<00:00, 11.55s/trial, best loss: 46.508074717803424]
{'D': 0, 'P': 2, 'Q': 2, 'd': 0, 'p': 1, 'q': 2, 'seasonal_order': 5}
Training .......
 model_WI_3_HOBBIES_2 
  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 3/3 [01:40<00:00, 33.62s/trial, best loss: 12.235408862502613]
{'D': 0, 'P': 0, 'Q': 1, 'd': 1, 'p': 0, 'q': 2, 'seasonal_order': 4}
Training .......
 model_WI_3_HOUSEHOLD_1 
 33%|███▎      | 1/3 [00:00<00:01,  1.23trial/s, best loss: 124.27539584461785]

100%|██████████| 3/3 [00:13<00:00,  4.46s/trial, best loss: 124.27539584461785]
{'D': 1, 'P': 0, 'Q': 1, 'd': 1, 'p': 0, 'q': 0, 'seasonal_order': 6}
Training .......
 model_WI_3_HOUSEHOLD_2 
100%|██████████| 3/3 [00:10<00:00,  3.54s/trial, best loss: 32.78613057630426]
{'D': 1, 'P': 0, 'Q': 2, 'd': 0, 'p': 2, 'q': 0, 'seasonal_order': 4}


### Save Best parameters

In [53]:
utils.save_to_json('sarimax_best_params',best,'w')

## Training with best parameters Model Training

### Reading Best Parameters

In [54]:
best =utils.read_from_json('sarimax_best_params',mode="r")
best.keys()

dict_keys(['best_param_CA_1_FOODS_1', 'best_param_CA_1_FOODS_2', 'best_param_CA_1_FOODS_3', 'best_param_CA_1_HOBBIES_1', 'best_param_CA_1_HOBBIES_2', 'best_param_CA_1_HOUSEHOLD_1', 'best_param_CA_1_HOUSEHOLD_2', 'best_param_CA_2_FOODS_1', 'best_param_CA_2_FOODS_2', 'best_param_CA_2_FOODS_3', 'best_param_CA_2_HOBBIES_1', 'best_param_CA_2_HOBBIES_2', 'best_param_CA_2_HOUSEHOLD_1', 'best_param_CA_2_HOUSEHOLD_2', 'best_param_CA_3_FOODS_1', 'best_param_CA_3_FOODS_2', 'best_param_CA_3_FOODS_3', 'best_param_CA_3_HOBBIES_1', 'best_param_CA_3_HOBBIES_2', 'best_param_CA_3_HOUSEHOLD_1', 'best_param_CA_3_HOUSEHOLD_2', 'best_param_CA_4_FOODS_1', 'best_param_CA_4_FOODS_2', 'best_param_CA_4_FOODS_3', 'best_param_CA_4_HOBBIES_1', 'best_param_CA_4_HOBBIES_2', 'best_param_CA_4_HOUSEHOLD_1', 'best_param_CA_4_HOUSEHOLD_2', 'best_param_TX_1_FOODS_1', 'best_param_TX_1_FOODS_2', 'best_param_TX_1_FOODS_3', 'best_param_TX_1_HOBBIES_1', 'best_param_TX_1_HOBBIES_2', 'best_param_TX_1_HOUSEHOLD_1', 'best_param_TX_

### Model Training

In [57]:
trained_model={}

for i in data.columns:

    y_train_data_ = pd.DataFrame()
    y_train_data_[i] = data[i].astype('float')
    
    order = (best[f"best_param_{i}"]['p'], best[f"best_param_{i}"]['d'], best[f"best_param_{i}"]['q'])
    seasonal_order = (best[f"best_param_{i}"]['P'], best[f"best_param_{i}"]['D'], best[f"best_param_{i}"]['Q'], 12)
    model = sm.tsa.statespace.SARIMAX(
        y_train_data_[i],
        order=order,
        seasonal_order=seasonal_order,
        initialization='approximate_diffuse'
    )
    print(f"Traning model_{i} ........")
    trained_model["model_{0}".format(i)] =model.fit()
    

Traning model_CA_1_FOODS_1 ........
Traning model_CA_1_FOODS_2 ........


Traning model_CA_1_FOODS_3 ........
Traning model_CA_1_HOBBIES_1 ........


Traning model_CA_1_HOBBIES_2 ........
Traning model_CA_1_HOUSEHOLD_1 ........


Traning model_CA_1_HOUSEHOLD_2 ........


Traning model_CA_2_FOODS_1 ........
Traning model_CA_2_FOODS_2 ........
Traning model_CA_2_FOODS_3 ........


...

Traning model_WI_1_HOUSEHOLD_2 ........
Traning model_WI_2_FOODS_1 ........
Traning model_WI_2_FOODS_2 ........
Traning model_WI_2_FOODS_3 ........


Traning model_WI_2_HOBBIES_1 ........
Traning model_WI_2_HOBBIES_2 ........


Traning model_WI_2_HOUSEHOLD_1 ........
Traning model_WI_2_HOUSEHOLD_2 ........
Traning model_WI_3_FOODS_1 ........


Traning model_WI_3_FOODS_2 ........
Traning model_WI_3_FOODS_3 ........
Traning model_WI_3_HOBBIES_1 ........


Traning model_WI_3_HOBBIES_2 ........
Traning model_WI_3_HOUSEHOLD_1 ........
Traning model_WI_3_HOUSEHOLD_2 ........


# Generate forcaste

In [59]:
"""
Generate a forecast for 100 days of each store 

"""
total_forecast =pd.DataFrame()
for i in trained_model.keys():
    forecast = trained_model[i].forecast(steps=100)
    total_forecast[i]=forecast

In [60]:
total_forecast.head()

,model_CA_1_FOODS_1,model_CA_1_FOODS_2,model_CA_1_FOODS_3,model_CA_1_HOBBIES_1,model_CA_1_HOBBIES_2,model_CA_1_HOUSEHOLD_1,model_CA_1_HOUSEHOLD_2,model_CA_2_FOODS_1,model_CA_2_FOODS_2,model_CA_2_FOODS_3,...,model_WI_2_HOBBIES_2,model_WI_2_HOUSEHOLD_1,model_WI_2_HOUSEHOLD_2,model_WI_3_FOODS_1,model_WI_3_FOODS_2,model_WI_3_FOODS_3,model_WI_3_HOBBIES_1,model_WI_3_HOBBIES_2,model_WI_3_HOUSEHOLD_1,model_WI_3_HOUSEHOLD_2
2016-04-25,342.307754,664.947651,2666.208185,520.967868,43.091422,957.246481,277.297283,467.060182,488.249205,1808.552064,...,37.779956,887.904640,146.754208,321.307356,517.227014,1863.223995,231.564485,31.005291,763.315111,170.666211
2016-04-26,349.875503,580.474222,2222.852710,483.502143,44.450147,759.402074,236.746997,456.436310,463.630944,1595.547300,...,37.898900,983.600808,120.673040,317.852484,517.026900,1840.803849,218.223409,31.277465,771.320621,138.741621
2016-04-27,363.255648,515.081560,2215.478268,484.749677,44.254515,715.957394,217.277376,456.436310,437.419353,1861.032175,...,37.993373,986.736226,114.283498,314.904983,583.973031,1962.318287,226.077284,31.225447,768.883123,142.742774
2016-04-28,361.539127,507.535999,2244.265113,488.957582,44.254858,692.454335,243.423824,456.436310,496.258486,1757.233524,...,37.900110,1063.992773,103.736986,311.051820,519.515401,1822.280248,228.880689,31.627842,766.932482,134.298730
2016-04-29,360.418126,492.889686,2155.955146,490.071283,42.790042,743.598570,291.730230,456.436310,488.032210,1818.477531,...,38.019297,1054.528368,116.133667,321.531685,542.114637,1865.280564,222.935865,30.984470,768.839784,132.677501


## Save forecast

In [61]:
total_forecast.to_csv("./forecasted_data/sarimax_forecast.csv", header=True)